# Consolidación de la base de datos meteorológica con todas las fuentes disponibles 

## Metodología de la consolidación

## Metodología para tratamiento de los datos en esta investigación para preparar los datos para el algoritmo SARIMAX  
1. Lista de atributos o variables meteorológicas. 
     * Temperatura máxima diaria (°C)
     * Temperatura mínima diaria (°C)
     * Temperatura media diaria (°C)
     * Humedad relativa media diaria (%)
     * Humedad específica diaria (kg/kg o g/kg)
     * Velocidad del viento media diaria (m/s)
     * Velocidad del viento máxima diaria (m/s)
     * Velocidad del viento mínima diaria (m/s)
     * Precipitación diaria (mm)
     * Días con lluvia por semana epidemiológica
     * Radiación ultravioleta diaria (UV index)
     * Presión atmosférica media diaria (hPa)

2. Lista de atributos o variables Epidemiológicas.
     - Casos diarios de dengue.
        - Número de casos de dengue en Caucasia (N_Casos_Total)
        - Número de casos de dengue por barrio (N_Casos_Barrio)

In [41]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

3. Obtener los datos meteorológicos: 
     - AMSC
     - NASAPOWER

Importación de los datos de AMSC


In [ ]:
direccion_viento = pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_direccion_102082706.csv", sep=',')
humedad= pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_humedad_102082706.csv", sep=',')
presion= pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_presion_102082706.csv", sep=',')
radiacion= pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_radiacion_102082706.csv", sep=',')
temperatura= pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_temperatura_102082706.csv", sep=',')
velocidad_viento= pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_velocidad_102082706.csv", sep=',')    
uv=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_uv_102082706.csv", sep=',')  
precipitacion=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\AMSC\Datos originales\AMSC_precipitacion_102082706.csv", sep=',')  


In [ ]:
from functools import reduce

# Diccionario de tus dataframes y los nombres de las variables
dataframes = {
    "direccion_viento": direccion_viento,
    "humedad": humedad,
    "precipitacion": precipitacion,
    "presion": presion,
    "radiacion": radiacion,
    "temperatura": temperatura,
    "uv": uv,
    "velocidad_viento": velocidad_viento
}

# Renombrar columnas 
for name, df in dataframes.items():
    cols = df.columns.tolist()
    if len(cols) > 1:
        df.rename(columns={cols[1]: name}, inplace=True)
    else:
        print(f"⚠️ El archivo {name} solo tiene la columna: {cols}. No se pudo renombrar la variable.")

# Unir todos los dataframes por la columna 'fecha'
df_final = reduce(lambda left, right: pd.merge(left, right, on="fecha", how="outer"), dataframes.values())

# Convertir 'fecha' a tipo datetime
df_final["fecha"] = pd.to_datetime(df_final["fecha"], errors="coerce")

# Ordenar por fecha ascendente
df_final = df_final.sort_values(by="fecha", ascending=True).reset_index(drop=True)


In [44]:
# Eliminar filas con al menos un dato faltante (NaN)
df_amsc = df_final.dropna()

# Revisar resultado
df_amsc


,fecha,direccion_viento,humedad,precipitacion,presion,radiacion,temperatura,uv,velocidad_viento
0,2022-10-04 04:58:00,220.5,68.0,0.0,1012.479515,525.5,29.138889,255.0,2.5
1,2022-10-04 04:59:00,102.0,80.0,0.0,1010.887912,277.5,26.583333,255.0,1.0
2,2022-10-04 05:00:00,110.5,80.0,0.0,1010.854048,277.5,26.583333,255.0,0.5
3,2022-10-04 05:01:00,116.0,80.0,0.0,1010.870980,277.5,26.611111,255.0,2.0
4,2022-10-04 05:02:00,110.0,80.0,0.0,1010.837116,495.5,26.638889,255.0,1.5
...,...,...,...,...,...,...,...,...,...
1563357,2025-09-23 20:55:00,5.0,82.0,0.0,1008.669828,0.0,28.833333,255.0,0.0
1563358,2025-09-23 20:56:00,5.0,82.0,0.0,1008.635964,0.0,28.833333,255.0,0.0
1563359,2025-09-23 20:57:00,5.0,82.0,0.0,1008.635964,0.0,28.833333,255.0,0.0
1563360,2025-09-23 20:58:00,5.0,83.0,0.0,1008.602100,0.0,28.833333,255.0,0.0


Ahora realizaremos un reemuestreo temporal de las variables climáticas, inicialmente a escala diaria y posteriormente a escala semanal (según el calendario epidemiológico de cada año), con el propósito de estandarizar la resolución temporal de todas las series y facilitar su análisis conjunto.
El tipo de agregación aplicado a cada variable se define según la naturaleza de sus mediciones:
- Dirección del viento: promedio diario y semanal (°)
- Humedad: promedio diario y semanal (%)
- Precipitación: acumulado diario y semanal (mm)
- Presión atmosférica: promedio diario y semanal (hPa)
- Radiación: promedio diario y semanal (W/m²)
- Temperatura: promedio diario y semanal (°C)
- Índice UV: promedio diario y semanal
- Velocidad del viento: promedio diario y semanal (m/s)

Importación de datos de la NASA POWER

In [45]:
# Importar datos
df_nasa=pd.read_csv(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Datos meteorológicos\NASA POWER\Datos_NS_2021-2024.csv", sep=';')
df_nasa

,YEAR,DOY,T2M,T2M_MAX,T2M_MIN,QV2M,RH2M,PRECTOTCORR,WS2M,WS2M_MAX,WS2M_MIN,WD2M,UV
0,2021,1,28.38,34.96,23.58,16.62,71.66,3.59,0.14,0.24,0.02,355.4,2.48
1,2021,2,27.44,33.39,23.64,17.98,80.54,12.12,0.10,0.23,0.04,294.9,2.18
2,2021,3,28.64,35.09,23.84,17.89,75.41,4.03,0.14,0.32,0.07,332.0,2.48
3,2021,4,28.48,35.05,23.69,15.91,68.69,0.70,0.14,0.25,0.07,329.4,2.47
4,2021,5,27.84,34.96,22.71,15.16,68.47,0.62,0.16,0.34,0.03,310.1,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2024,362,25.83,28.78,23.57,18.48,88.70,2.16,0.19,0.46,0.04,300.9,1.79
1457,2024,363,26.05,29.56,23.60,19.30,91.31,9.07,0.14,0.31,0.06,321.3,1.51
1458,2024,364,24.92,26.19,23.93,19.24,96.95,8.35,0.15,0.38,0.03,101.8,1.70
1459,2024,365,25.11,27.89,23.30,18.50,92.34,1.89,0.13,0.36,0.01,250.0,1.65


4. Renombrar los atributos por fuente.

In [46]:
# renombrar columnas del df final_sin_na
df_amsc.rename(columns={
    'temperatura': 'Temp (AMSC)',
    'humedad': 'HumRel (AMSC)',
    'precipitacion': 'Prec (AMSC)',
    'velocidad_viento': 'VelV (AMSC)',
    'direccion_viento': 'DirV (AMSC)',
    'presion': 'Pres (AMSC)',
    'radiacion': 'Rad (AMSC)',
    'uv': 'Uv (AMSC)'
}, inplace=True)

df_amsc


C:\Users\usuario1\AppData\Local\Temp\ipykernel_6824\2581485777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_amsc.rename(columns={


,fecha,DirV (AMSC),HumRel (AMSC),Prec (AMSC),Pres (AMSC),Rad (AMSC),Temp (AMSC),Uv (AMSC),VelV (AMSC)
0,2022-10-04 04:58:00,220.5,68.0,0.0,1012.479515,525.5,29.138889,255.0,2.5
1,2022-10-04 04:59:00,102.0,80.0,0.0,1010.887912,277.5,26.583333,255.0,1.0
2,2022-10-04 05:00:00,110.5,80.0,0.0,1010.854048,277.5,26.583333,255.0,0.5
3,2022-10-04 05:01:00,116.0,80.0,0.0,1010.870980,277.5,26.611111,255.0,2.0
4,2022-10-04 05:02:00,110.0,80.0,0.0,1010.837116,495.5,26.638889,255.0,1.5
...,...,...,...,...,...,...,...,...,...
1563357,2025-09-23 20:55:00,5.0,82.0,0.0,1008.669828,0.0,28.833333,255.0,0.0
1563358,2025-09-23 20:56:00,5.0,82.0,0.0,1008.635964,0.0,28.833333,255.0,0.0
1563359,2025-09-23 20:57:00,5.0,82.0,0.0,1008.635964,0.0,28.833333,255.0,0.0
1563360,2025-09-23 20:58:00,5.0,83.0,0.0,1008.602100,0.0,28.833333,255.0,0.0


In [48]:
# Renombrar columnas del dataframe nasa
df_nasa.rename(columns={
    'YEAR': 'Año', 'DOY': 'Dia',
    'T2M': 'Temp (NASA)',
    'T2M_MAX': 'Temp_Max (NASA)',
    'T2M_MIN': 'Temp_Min (NASA)',
    'QV2M': 'HumEsp (NASA)',
    'RH2M': 'HumRel (NASA)',
    'PRECTOTCORR': 'Prec (NASA)',
    'WS2M': 'VelV (NASA)',
    'WS2M_MAX': 'VelV_Max (NASA)',
    'WS2M_MIN': 'VelV_Min (NASA)',
    'WD2M': 'DirV (NASA)',
    'UV': 'Uv (NASA)'
}, inplace=True)
df_nasa

,Año,Dia,Temp (NASA),Temp_Max (NASA),Temp_Min (NASA),HumEsp (NASA),HumRel (NASA),Prec (NASA),VelV (NASA),VelV_Max (NASA),VelV_Min (NASA),DirV (NASA),Uv (NASA)
0,2021,1,28.38,34.96,23.58,16.62,71.66,3.59,0.14,0.24,0.02,355.4,2.48
1,2021,2,27.44,33.39,23.64,17.98,80.54,12.12,0.10,0.23,0.04,294.9,2.18
2,2021,3,28.64,35.09,23.84,17.89,75.41,4.03,0.14,0.32,0.07,332.0,2.48
3,2021,4,28.48,35.05,23.69,15.91,68.69,0.70,0.14,0.25,0.07,329.4,2.47
4,2021,5,27.84,34.96,22.71,15.16,68.47,0.62,0.16,0.34,0.03,310.1,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2024,362,25.83,28.78,23.57,18.48,88.70,2.16,0.19,0.46,0.04,300.9,1.79
1457,2024,363,26.05,29.56,23.60,19.30,91.31,9.07,0.14,0.31,0.06,321.3,1.51
1458,2024,364,24.92,26.19,23.93,19.24,96.95,8.35,0.15,0.38,0.03,101.8,1.70
1459,2024,365,25.11,27.89,23.30,18.50,92.34,1.89,0.13,0.36,0.01,250.0,1.65


5. Ordenar los datos meteorológicos cronológicamente en formato DD-MM-AAAA
   * Primero ordenar los datos de la NASA juntando los atributos de día y año en un o solo llamado fecha.
   * 

In [49]:
# Crear columna 'fecha' a partir de 'Año' y 'Dia'
df_nasa_fecha = df_nasa.copy()
df_nasa_fecha['fecha'] = pd.to_datetime(df_nasa_fecha['Año'].astype(str) + '-' + df_nasa_fecha['Dia'].astype(str), format='%Y-%j')
df_nasa_fecha = df_nasa_fecha.drop(columns=['Año', 'Dia'])
# Reordenar columnas para que 'fecha' sea la primera
df_nasa_fecha = df_nasa_fecha[['fecha'] + [col for col in df_nasa_fecha.columns if col != 'fecha']]
df_nasa_fecha


,fecha,Temp (NASA),Temp_Max (NASA),Temp_Min (NASA),HumEsp (NASA),HumRel (NASA),Prec (NASA),VelV (NASA),VelV_Max (NASA),VelV_Min (NASA),DirV (NASA),Uv (NASA)
0,2021-01-01,28.38,34.96,23.58,16.62,71.66,3.59,0.14,0.24,0.02,355.4,2.48
1,2021-01-02,27.44,33.39,23.64,17.98,80.54,12.12,0.10,0.23,0.04,294.9,2.18
2,2021-01-03,28.64,35.09,23.84,17.89,75.41,4.03,0.14,0.32,0.07,332.0,2.48
3,2021-01-04,28.48,35.05,23.69,15.91,68.69,0.70,0.14,0.25,0.07,329.4,2.47
4,2021-01-05,27.84,34.96,22.71,15.16,68.47,0.62,0.16,0.34,0.03,310.1,2.30
...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2024-12-27,25.83,28.78,23.57,18.48,88.70,2.16,0.19,0.46,0.04,300.9,1.79
1457,2024-12-28,26.05,29.56,23.60,19.30,91.31,9.07,0.14,0.31,0.06,321.3,1.51
1458,2024-12-29,24.92,26.19,23.93,19.24,96.95,8.35,0.15,0.38,0.03,101.8,1.70
1459,2024-12-30,25.11,27.89,23.30,18.50,92.34,1.89,0.13,0.36,0.01,250.0,1.65


6. Obtener los datos epidemiológicos

In [56]:
# Cargar datos epidemiológicos 
df_casos = pd.read_excel(r"C:\Users\usuario1\OneDrive - Universidad de Antioquia\UNIVERSIDAD DE ANTIOQUIA\Proyecto SAT Dengue\Bases de datos\Secretaria de salud\BD_DENGUE_SEC_SALUD_2021-2024.xlsx")
df_casos

,cod_eve,fec_not,semana,año,cod_pre,cod_sub,edad_,uni_med_,nacionali_,nombre_nacionalidad,...,nom_upgd,npais_proce,ndep_proce,nmun_proce,npais_resi,ndep_resi,nmun_resi,ndep_notif,nmun_notif,nreg
0,210,2021-01-11,1,2021,515402201,1,40,1,170,COLOMBIA,...,ESE HOSPITAL CESAR URIBE PIEDRAHITA,COLOMBIA,ANTIOQUIA,EL BAGRE,COLOMBIA,ANTIOQUIA,EL BAGRE,ANTIOQUIA,CAUCASIA,1800
1,210,2021-02-06,5,2021,515402201,1,12,1,170,COLOMBIA,...,ESE HOSPITAL CESAR URIBE PIEDRAHITA,COLOMBIA,ANTIOQUIA,ZARAGOZA,COLOMBIA,ANTIOQUIA,ZARAGOZA,ANTIOQUIA,CAUCASIA,1791
2,210,2021-02-12,6,2021,2548885069,80,25,1,170,COLOMBIA,...,SIN INFORMACIÓN,COLOMBIA,CUNDINAMARCA,NILO,COLOMBIA,ANTIOQUIA,CAUCASIA,SIN INFORMACIÓN,SIN INFORMACIÓN,1798
3,210,2021-02-16,3,2021,526605568,10,38,1,170,COLOMBIA,...,SIN INFORMACIÓN,COLOMBIA,ANTIOQUIA,CAUCASIA,COLOMBIA,ANTIOQUIA,ENVIGADO,SIN INFORMACIÓN,SIN INFORMACIÓN,1793
4,210,2021-02-19,6,2021,526605568,10,54,1,170,COLOMBIA,...,SIN INFORMACIÓN,COLOMBIA,ANTIOQUIA,CAUCASIA,COLOMBIA,BOGOTA,BOGOTA,SIN INFORMACIÓN,SIN INFORMACIÓN,1795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5485,210,2024-12-30,52,2024,515402201,1,6,1,170,COLOMBIA,...,ESE HOSPITAL CESAR URIBE PIEDRAHITA,COLOMBIA,ANTIOQUIA,CAUCASIA,COLOMBIA,ANTIOQUIA,CAUCASIA,ANTIOQUIA,CAUCASIA,2025-01-01 00:00:00
5486,210,2024-12-31,49,2024,515415510,1,9,1,170,COLOMBIA,...,BIOMED VIDA SAS,COLOMBIA,ANTIOQUIA,ZARAGOZA,COLOMBIA,ANTIOQUIA,CAUCASIA,ANTIOQUIA,CAUCASIA,2024-12-31 00:00:00
5487,210,2024-12-31,52,2024,515415510,1,42,1,170,COLOMBIA,...,BIOMED VIDA SAS,COLOMBIA,ANTIOQUIA,MEDELLIN,COLOMBIA,ANTIOQUIA,CAUCASIA,ANTIOQUIA,CAUCASIA,2024-12-31 00:00:00
5488,210,2024-12-31,52,2024,515403817,1,2,1,170,COLOMBIA,...,CLINICA PAJONAL LIMITADA,COLOMBIA,ANTIOQUIA,CACERES,COLOMBIA,ANTIOQUIA,CACERES,ANTIOQUIA,CAUCASIA,2025-01-07 00:00:00


7. Remuestreartrear o hacer resampling. 
      - Los datos obtenidos de AMSC se encuentran a escala por minuto, se deben convertir a escala diaria para luego fusionarlos con los datos de NASAPOWER y por último convertirlos a escala semana epidemiológica.